In [72]:
from pymtl3 import *
from pymtl3.passes.backends.verilog import *

In [73]:
# Simulation
class Adder(Component):
    def construct (s):
        s.a = InPort(32)
        s.b = InPort(32)
        s.q = OutPort(32)

        @update
        def comb():
            s.q @= s.a + s.b

In [74]:
a = Adder()
a.apply(DefaultPassGroup(linetrace=True, textwave=True))
a.sim_reset()

a.a @= 5
a.b @= 7
a.sim_eval_combinational()
print(a.q.uint())

12


In [75]:
a.sim_tick()
a.print_textwave()


      |0    |1    |2    |3    

  clk /‾‾\__/‾‾\__/‾‾\__/‾‾\__

reset ‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾\_____

    a  0000000          |+0005

    b  0000000          |+0007

    q  0000000          |+000c

